Recursos:
 - https://docs.datastax.com/en/developer/python-driver/3.25/getting_started/
 - https://cassandra.apache.org/doc/latest/cassandra/faq/
 - https://openwebinars.net/blog/como-usar-apache-cassandra-con-python/
 - https://docs.datastax.com/en/cql-oss/3.3/cql/cql_reference/cqlCreateTable.html
 - https://www.datastax.com/blog/most-important-thing-know-cassandra-data-modeling-primary-key

## Testeando la conexión a cassandra y algunas queries

In [1]:
import pandas as pd

In [2]:
from cassandra.cluster import Cluster

#Conexion al cluster
#Es importante haber mapeado el puerto usando -p 9042:9042 al crear el contenedor

cluster = Cluster(['127.0.0.1'], port=9042)
#session = cluster.connect('bdnosql') #El keyspace lo creé desde la terminal

---------

## Creación de keyspace y tablas

In [3]:
session = cluster.connect()

session.execute(
    """
    DROP KEYSPACE IF EXISTS bd_libros;
    """
)

# Creación del keyspace
session.execute(
    """
    CREATE KEYSPACE bd_libros 
    WITH replication = {'class' : 'SimpleStrategy', 'replication_factor':1};
    """
)
session.set_keyspace('bd_libros')


### Creación de las tablas

# LIBROS POR CLIENTE (Queries 1,2,3)
session.execute(
    """
    CREATE TABLE libros_por_cliente (
    id_cliente text,
    titulo_libro text,
    autor_libro text,
    categoria text,
    calificacion int,
    nombre_cliente text STATIC,
    pais text STATIC,
    membresia text STATIC,
    PRIMARY KEY(id_cliente, titulo_libro, autor_libro)
    );
    """
)

## CATEGORIAS POR CLIENTE
#session.execute(
#    """
#    CREATE MATERIALIZED VIEW categorias_por_cliente AS
#    SELECT id_cliente, categoria, titulo_libro, autor_libro, calificacion
#    FROM libros_por_cliente
#    WHERE categoria IS NOT NULL
#    AND titulo_libro IS NOT NULL
#    AND autor_libro IS NOT NULL
#    PRIMARY KEY(id_cliente, categoria, titulo_libro, autor_libro);
#    """
#)

#CATEGORIAS POR CLIENTE
session.execute(
    """
    CREATE TABLE categorias_por_cliente(
    id_cliente text,    
    categoria text,
    titulo_libro text,
    autor_libro text,
    calificacion int,
    PRIMARY KEY(id_cliente,categoria, titulo_libro, autor_libro, calificacion)
    );
    """
)


# CLIENTES POR LIBRO (Query 4)
session.execute(
    """
    CREATE TABLE clientes_por_libro (
    titulo_libro text,
    autor_libro text,
    calificacion int,
    id_cliente text,
    categoria text,
    PRIMARY KEY((titulo_libro, autor_libro), calificacion, id_cliente)
    );
    """
)


# LIBROS POR CATEGORIA (Query 5)
session.execute(
    """
    CREATE TABLE libros_por_categoria (
    categoria text,
    titulo_libro text,
    autor_libro text,
    calificacion_promedio float,
    PRIMARY KEY (categoria, titulo_libro, autor_libro)
    );
    """
)
#session.execute(
#    """
#    CREATE INDEX ON libros_por_categoria (titulo_libro);
#    """
#)
#session.execute(
#    """
#    CREATE INDEX ON libros_por_categoria (autor_libro);
#    """
#)

# CALIFICACIONES POR CATEGORÍA
session.execute(
    """
    CREATE TABLE calificaciones_por_categoria(
    categoria text,
    calificacion_promedio float,
    titulo_libro text,
    autor_libro text,
    PRIMARY KEY(categoria, calificacion_promedio, titulo_libro, autor_libro)
    )
    WITH CLUSTERING ORDER BY (calificacion_promedio DESC);
    """
)

## Queries

In [4]:
def Q0_check_usuario(id_cliente:str):
    existe_usuario = session.execute(
        """
        SELECT * FROM libros_por_cliente
        WHERE id_cliente = %s
        LIMIT 1
        """,
        [id_cliente]
    )
    if existe_usuario: 
        return True
    else: 
        return False

In [5]:
def Q1_alta_usuario(id_cliente:str, nombre_cliente:str, pais:str, membresia:str):
    existe_usuario = session.execute(
        """
        SELECT * FROM libros_por_cliente
        WHERE id_cliente = %s
        LIMIT 1
        """,
        [id_cliente]
    )

    if existe_usuario:
        return None

    session.execute(
        """
        INSERT INTO libros_por_cliente
        (id_cliente, nombre_cliente, pais, membresia)
        VALUES (%s, %s, %s, %s)
        """,
        (id_cliente, nombre_cliente, pais, membresia)
    )
    return 

In [6]:
def Q2_agregar_calificacion(id_cliente:str , titulo_libro:str, autor_libro:str, categoria:str, calificacion:int):
    existe_calif= session.execute(
        """
        SELECT categoria, calificacion FROM libros_por_cliente
        WHERE id_cliente=%s AND titulo_libro=%s  AND autor_libro=%s
        LIMIT 1
        """,
        [id_cliente, titulo_libro, autor_libro]
    )
    info_previa = existe_calif.one()
    
    if existe_calif:
        # pedir a usuario confirma que desea cambiar la calificación
        confirmacion_del_usuario = 1 #respuesta del usuario
        if not confirmacion_del_usuario:
            return

        info_previa = existe_calif.one()
        
        #Borrar la calificación anterior
        session.execute(
            """
            DELETE FROM clientes_por_libro
            WHERE titulo_libro=%s
            AND autor_libro=%s
            AND calificacion=%s
            AND id_cliente=%s
            """,
            (titulo_libro, autor_libro, info_previa.calificacion, id_cliente)
        )
        session.execute(
            """
            DELETE FROM categorias_por_cliente
            WHERE id_cliente=%s 
            AND categoria=%s
            AND titulo_libro=%s
            AND autor_libro=%s
            AND calificacion=%s
            """,
            (id_cliente, info_previa.categoria, titulo_libro, autor_libro, info_previa.calificacion)
        )
        

    #Agregar la nueva calificación
    session.execute(
        """
        INSERT INTO libros_por_cliente 
        (id_cliente, titulo_libro, autor_libro, categoria, calificacion)
        VALUES (%s, %s, %s, %s, %s)
        """,
        (id_cliente, titulo_libro, autor_libro, categoria, calificacion)
    )
    session.execute(
        """
        INSERT INTO clientes_por_libro 
        (titulo_libro, autor_libro, id_cliente, calificacion, categoria)
        VALUES (%s, %s, %s, %s, %s)
        """,
        (titulo_libro, autor_libro, id_cliente, calificacion, categoria)
    )
    session.execute(
        """
        INSERT INTO categorias_por_cliente 
        (id_cliente, titulo_libro, autor_libro, categoria, calificacion)
        VALUES (%s, %s, %s, %s, %s)
        """,
        (id_cliente, titulo_libro, autor_libro, categoria, calificacion)
    )
    

    #Actualizar calificaciones promedio
    calificacion_promedio = session.execute(
        """
        SELECT AVG(calificacion) FROM clientes_por_libro
        WHERE titulo_libro=%s
        AND autor_libro=%s
        """,
        (titulo_libro, autor_libro)
    )
    calificacion_promedio = round(calificacion_promedio.one()[0], 2)

    categorias_del_libro = session.execute(
        """
        SELECT categoria FROM clientes_por_libro
        WHERE titulo_libro =%s
        AND autor_libro = %s
        """,
        (titulo_libro, autor_libro)
    )

    alguna_categoria = categorias_del_libro.one().categoria

    promedio_anterior = session.execute(
        """
        SELECT calificacion_promedio FROM libros_por_categoria
        WHERE categoria =%s
        AND titulo_libro =%s
        AND autor_libro = %s
        """,
        (alguna_categoria, titulo_libro, autor_libro)
    )
    if promedio_anterior:
        promedio_anterior = promedio_anterior.one().calificacion_promedio
    
    #En caso de que sea la única review que asigna esa categoría al libro y se cambie la categoría, necesitamos
    #eliminar la fila de la tabla libros_por_categoría
    if info_previa is not None:
        if  info_previa.categoria != categoria:
            unica_calificacion = True
    
            for row in categorias_del_libro:
                if row.categoria == info_previa.categoria: #entonces hay otras reviews que categorizan igual al libro
                    unica_calificacion = False
                
                #Actualizar libros_por_categoria
                session.execute(
                    """
                    INSERT INTO libros_por_categoria
                    (categoria, titulo_libro, autor_libro, calificacion_promedio)
                    VALUES (%s, %s, %s, %s)
                    """,
                    (row.categoria, titulo_libro, autor_libro, calificacion_promedio)
                )

                #Actualizar calificaciones_por_categoria
                ## Borrar promedio anterior
                if promedio_anterior:
                    session.execute(
                        """
                        DELETE FROM calificaciones_por_categoria
                        WHERE categoria=%s
                        AND calificacion_promedio=%s
                        AND titulo_libro=%s
                        AND autor_libro= %s
                        """,
                        (row.categoria, promedio_anterior, titulo_libro, autor_libro)
                    )
                ## Insertar promedio nuevo
                session.execute(
                    """
                    INSERT INTO calificaciones_por_categoria
                    (categoria, calificacion_promedio, titulo_libro, autor_libro)
                    VALUES (%s, %s, %s, %s)
                    """,
                    (row.categoria, calificacion_promedio, titulo_libro, autor_libro)
                )
                
            
            if unica_calificacion:
                session.execute(
                    """
                    DELETE FROM libros_por_categoria
                    WHERE categoria=%s
                    AND titulo_libro=%s
                    AND autor_libro=%s
                    """,
                    (info_previa.categoria, titulo_libro, autor_libro)
                )
                session.execute(
                    """
                    DELETE FROM calificaciones_por_categoria
                    WHERE categoria=%s
                    AND calificacion_promedio=%s
                    AND titulo_libro=%s
                    AND autor_libro=%s
                    """,
                    (info_previa.categoria,promedio_anterior, titulo_libro, autor_libro)

                )
    
    else:
        for row in categorias_del_libro:
            #Actualizar libros_por_categoria
            session.execute(
                """
                INSERT INTO libros_por_categoria
                (categoria, titulo_libro, autor_libro, calificacion_promedio)
                VALUES (%s, %s, %s, %s)
                """,
                (row.categoria, titulo_libro, autor_libro, calificacion_promedio)
            )

            #Actualizar calificaciones_por_categoria
            ## Borrar promedio anterior
            if promedio_anterior:
                session.execute(
                    """
                    DELETE FROM calificaciones_por_categoria
                    WHERE categoria=%s
                    AND calificacion_promedio=%s
                    AND titulo_libro=%s
                    AND autor_libro= %s
                    """,
                    (row.categoria, promedio_anterior, titulo_libro, autor_libro)
                )
            #Insertar promedio nuevo
            session.execute(
                """
                INSERT INTO calificaciones_por_categoria
                (categoria, calificacion_promedio, titulo_libro, autor_libro)
                VALUES (%s, %s, %s, %s)
                """,
                (row.categoria, calificacion_promedio, titulo_libro, autor_libro)
            )

    return 

In [7]:
def Q3_categoría_preferida_por_cliente(id_cliente:str):
    
    resultado = session.execute(
        """
        SELECT categoria, AVG(calificacion) FROM categorias_por_cliente
        WHERE id_cliente=%s
        GROUP BY categoria
        """,
        [id_cliente]
    )

    resultado = pd.DataFrame(resultado.all(), columns=['cateogoria', 'avg_calificacion'])
    resultado = resultado.sort_values(by='avg_calificacion', ascending=False)
    resultado = resultado.values[0]

    return (resultado[0], resultado[1])

In [15]:
def Q4_mas_disfrutaron_libro(titulo_libro:str, autor_libro:str):
    
    resultado_max_calif = session.execute(
        """
        SELECT MAX(calificacion) FROM clientes_por_libro
        WHERE titulo_libro=%s
        AND autor_libro=%s
        LIMIT 1
        """,
        (titulo_libro, autor_libro)
    )

    max_calif = resultado_max_calif.one()[0]

    if max_calif is None:
        return None

    resultado_clientes = session.execute(
        """
        SELECT id_cliente, calificacion FROM clientes_por_libro
        WHERE titulo_libro=%s
        AND autor_libro=%s
        AND calificacion=%s
        """, 
        (titulo_libro, autor_libro, max_calif)
    )
        
    return pd.DataFrame(resultado_clientes.all())

In [9]:
# Libro con mejor calificacion promedio de una categoria dada
def Q5_mejores_libros_por_categoria(categoria:str, n:int=10):
    
    res = session.execute(
        """
        SELECT titulo_libro, autor_libro, calificacion_promedio FROM calificaciones_por_categoria
        WHERE categoria=%s
        ORDER BY calificacion_promedio DESC
        LIMIT %s;
        """,
        (categoria, n)
    )

    return pd.DataFrame(res.all())

In [10]:
def Q6_user_libros(id_cliente: str): 
    resultado = session.execute(
        """
        SELECT titulo_libro, autor_libro, categoria, calificacion FROM libros_por_cliente
        WHERE id_cliente=%s
        """,
        [id_cliente]
    )

    return pd.DataFrame(resultado.all())


## Testing las queries

In [11]:
# Dar de alta usuarios
Q1_alta_usuario('Roger21', 'Rogelio Mendez', 'México', 'Estudiante')
Q1_alta_usuario('ProfPimentel', 'Alejandro Pimentel', 'Francia', 'Profesor')
Q1_alta_usuario('Pily01', 'Pilar Angeles', 'Francia', 'Profesor')

# Añadir libros y calificaciones
Q2_agregar_calificacion('ProfPimentel', 'Duna', 'Frank Herbert', 'Ciencia Ficción', 5)
Q2_agregar_calificacion('ProfPimentel', 'Duna 2', 'Frank Herbert', 'Ciencia Ficción', 1)
Q2_agregar_calificacion('Roger21', 'Duna', 'Frank Herbert', 'Ciencia Ficción', 3)
Q2_agregar_calificacion('Roger21', 'Duna', 'Frank Herbert', 'Ciencia Ficción', 4)
Q2_agregar_calificacion('Pily01', 'Duna', 'Frank Herbert', 'Ciencia Ficción', 5)
Q2_agregar_calificacion('Pily01', 'La Máquina del Tiempo', 'Herbert George Wells', 'Ciencia Ficción', 5)
Q2_agregar_calificacion('ProfPimentel', 'El psicoanalista', 'John Katzenbach', 'Thriller', 5)

In [16]:
res = session.execute(
        """
        SELECT * FROM libros_por_cliente
        """
    )
res = pd.DataFrame(res.all())
res

,id_cliente,titulo_libro,autor_libro,membresia,nombre_cliente,pais,calificacion,categoria
0,Pily01,Duna,Frank Herbert,Profesor,Pilar Angeles,Francia,5,Ciencia Ficción
1,Pily01,La Máquina del Tiempo,Herbert George Wells,Profesor,Pilar Angeles,Francia,5,Ciencia Ficción
2,ProfPimentel,Duna,Frank Herbert,Profesor,Alejandro Pimentel,Francia,5,Ciencia Ficción
3,ProfPimentel,Duna 2,Frank Herbert,Profesor,Alejandro Pimentel,Francia,1,Ciencia Ficción
4,ProfPimentel,El psicoanalista,John Katzenbach,Profesor,Alejandro Pimentel,Francia,5,Thriller
5,Roger21,Duna,Frank Herbert,Estudiante,Rogelio Mendez,México,4,Ciencia Ficción


In [17]:
res = session.execute(
        """
        SELECT * FROM clientes_por_libro
        """
    )
res = pd.DataFrame(res.all())
res

,titulo_libro,autor_libro,calificacion,id_cliente,categoria
0,El psicoanalista,John Katzenbach,5,ProfPimentel,Thriller
1,Duna,Frank Herbert,4,Roger21,Ciencia Ficción
2,Duna,Frank Herbert,5,Pily01,Ciencia Ficción
3,Duna,Frank Herbert,5,ProfPimentel,Ciencia Ficción
4,Duna 2,Frank Herbert,1,ProfPimentel,Ciencia Ficción
5,La Máquina del Tiempo,Herbert George Wells,5,Pily01,Ciencia Ficción


In [18]:
res = session.execute(
        """
        SELECT * FROM libros_por_categoria
        """
    )
res = pd.DataFrame(res.all())
res

,categoria,titulo_libro,autor_libro,calificacion_promedio
0,Ciencia Ficción,Duna,Frank Herbert,4.0
1,Ciencia Ficción,Duna 2,Frank Herbert,1.0
2,Ciencia Ficción,La Máquina del Tiempo,Herbert George Wells,5.0
3,Thriller,El psicoanalista,John Katzenbach,5.0


In [20]:
# CONSULTANDO LAS TABLAS
res0 = Q0_check_usuario('ProfPimentel')
print(res0)

res1 = Q3_categoría_preferida_por_cliente('ProfPimentel')
print(res1)

res2 = Q4_mas_disfrutaron_libro('Duna', 'Frank Herbert')
print(res2)

res3 = Q5_mejores_libros_por_categoria('Ciencia Ficción')
print(res3)

('Thriller', 5)
     id_cliente  calificacion
0        Pily01             5
1  ProfPimentel             5
            titulo_libro           autor_libro  calificacion_promedio
0  La Máquina del Tiempo  Herbert George Wells                    5.0
1                   Duna         Frank Herbert                    4.0
2                 Duna 2         Frank Herbert                    1.0


## Cargando nuestros datos

In [20]:
reviews = pd.read_csv('resources/data.csv')
users=pd.read_csv("resources/users.csv")

In [21]:
reviews.head(5)

,id_cliente,titulo_libro,autor_libro,categoria,calificacion
0,pabs,To Kill A Mockingbird,Harper Lee,Novela,8
1,laz38,1984,George Orwell,Thriller,8
2,laz38,Doña Perfecta,Benito Perez,Realismo mágico,6
3,guillexus,Anna Karenina,Leon Tolstoi,Novela,5
4,lucluc,La guerra de los mundos,H. G. Wells,Novela,4


In [22]:
#Notemos que Pilar y Prof Pimentel no se agragaron a nuestro df de usuarios ya que aún no existía el df
users.tail(5)

,id_cliente,nombre_cliente,pais,membresia
13,lucluc,Lucas,Ecuador,Profesor
14,MarceCL,Marcela,Mexico,Estudiante
15,OctaH,Octavio,Estados Unidos,Estudiante
16,pabs,Pablo,Chile,Premium
17,Yeu,Yeudiel,Francia,Profesor


In [23]:
#Cargamos nuestros datos
for index, row in users.iterrows():
        Q1_alta_usuario(row[0], row[1], row[2], row[3])

for index, row in reviews.iterrows():
    Q2_agregar_calificacion(row[0], row[1], row[2], row[3], row[4])

In [24]:
#Vemos que nuestra base se actualizó correctamente
res = session.execute(
        """
        SELECT * FROM libros_por_cliente
        """
    )
res = pd.DataFrame(res.all())
res

,id_cliente,titulo_libro,autor_libro,membresia,nombre_cliente,pais,calificacion,categoria
0,laz38,100 años de soledad,Gabriel Garcia,Estudiante,Laz,Brasil,9,Cuento
1,laz38,1984,George Orwell,Estudiante,Laz,Brasil,8,Thriller
2,laz38,Amor en tiempos de Colera,Gabriel Garcia,Estudiante,Laz,Brasil,6,Misterio
3,laz38,Anna Karenina,Leon Tolstoi,Estudiante,Laz,Brasil,9,Drama
4,laz38,Bajo la rueda,Hermann Hesse,Estudiante,Laz,Brasil,8,Thriller histórico
...,...,...,...,...,...,...,...,...
1158,kikin01,Un hogar solido,Elena Garro,Profesor,Enrique,Japon,3,Tragedia
1159,kikin01,Un mundo feliz,Aldous Huxley,Profesor,Enrique,Japon,8,Ciencia Ficcion
1160,kikin01,Una educación,Tara Westlover,Profesor,Enrique,Japon,6,Académico
1161,kikin01,Valis,K. Dick,Profesor,Enrique,Japon,5,Thriller histórico


In [25]:
res = session.execute(
        """
        SELECT * FROM libros_por_categoria
        """
    )
res = pd.DataFrame(res.all())
res

,categoria,titulo_libro,autor_libro,calificacion_promedio
0,Ciencia Ficción,Duna,Frank Herbert,4.0
1,Ciencia Ficción,Duna 2,Frank Herbert,1.0
2,Ciencia Ficción,La Máquina del Tiempo,Herbert George Wells,5.0
3,Ficción Doméstica,1984,George Orwell,5.0
4,Ficción Doméstica,Anna Karenina,Leon Tolstoi,7.0
...,...,...,...,...
911,Realismo mágico,The Lord of the Rings The Two Towers,Tolkien,5.0
912,Realismo mágico,The Pillars of the Earth,Ken Follett,5.0
913,Realismo mágico,The tell-tale heart,Edgar Allan Poe,4.0
914,Realismo mágico,To Kill A Mockingbird,Harper Lee,6.0


---
## Probamos las queries

In [29]:
#Probamos nuestras Querys
res1 = Q3_categoría_preferida_por_cliente('MarceCL')
res1

('Fantasia', 10)

In [30]:
res2 = Q4_mas_disfrutaron_libro('The tell-tale heart', 'Edgar Allan Poe')
res2

,id_cliente,calificacion
0,guillexus,9


In [31]:

res3 = Q5_mejores_libros_por_categoria('Ciencia Ficción')
res3

,titulo_libro,autor_libro,calificacion_promedio
0,La Máquina del Tiempo,Herbert George Wells,5.0
1,Duna,Frank Herbert,4.0
2,Duna 2,Frank Herbert,1.0


In [1]:
import pandas as pd
import numpy as np
import glob
import os

os.chdir("C:/Users/Pabul/Desktop/databike")

"""extension="csv"
all_files= [i for i in glob.glob('*.{}'.format(extension))]
combined=pd.concat([pd.read_csv(i) for i in all_files])
combined.to_csv("comb.csv", index=  False, encoding="utf-8")"""

df=pd.read_csv("comb.csv")
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,634,2013-07-01 00:00:00,2013-07-01 00:10:34,164,E 47 St & 2 Ave,40.753231,-73.970325,504,1 Ave & E 15 St,40.732219,-73.981656,16950,Customer,\N,0
1,1547,2013-07-01 00:00:02,2013-07-01 00:25:49,388,W 26 St & 10 Ave,40.749718,-74.002950,459,W 20 St & 11 Ave,40.746745,-74.007756,19816,Customer,\N,0
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730287,-73.990765,237,E 11 St & 2 Ave,40.730473,-73.986724,14548,Subscriber,1980,2
3,1580,2013-07-01 00:01:06,2013-07-01 00:27:26,531,Forsyth St & Broome St,40.718939,-73.992663,499,Broadway & W 60 St,40.769155,-73.981918,16063,Customer,\N,0
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.734927,-73.992005,410,Suffolk St & Stanton St,40.720664,-73.985180,19213,Subscriber,1986,1


In [19]:
estaciones=df[['start station longitude', 'start station latitude',"start station id", 'start station name']]
estaciones.drop_duplicates(inplace=True)
estaciones.to_csv('estaciones.csv', index=  False, encoding="utf-8")
estaciones

C:\Users\Pabul\AppData\Local\Temp\ipykernel_20004\2930951937.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estaciones.drop_duplicates(inplace=True)


,start station longitude,start station latitude,start station id,start station name
0,-73.970325,40.753231,164,E 47 St & 2 Ave
1,-74.002950,40.749718,388,W 26 St & 10 Ave
2,-73.990765,40.730287,293,Lafayette St & E 8 St
3,-73.992663,40.718939,531,Forsyth St & Broome St
4,-73.992005,40.734927,382,University Pl & E 14 St
...,...,...,...,...
12443,-73.958089,40.694528,372,Franklin Ave & Myrtle Ave
17788,-73.978034,40.696089,119,Park Ave & St Edwards St
140642,-73.971214,40.750224,2017,E 43 St & 2 Ave
301066,-74.002347,40.721655,2010,Grand St & Greene St


In [ ]:
viajes=df[["bikeid", ""]]